# Chocolate Milkshake Betaseries Analysis

In [80]:
import glob
import os
from os import listdir
from shutil import rmtree
from subprocess import check_output
import argparse
import re

## Setup feat files

In [2]:
def make_file(sub_id, ses_id, trial_id, output_dir, task,data_dict):
    with open(os.path.join('/projects/niblab/experiments/chocolate_milkshake/data/code/beta_design.fsf'),'r') as infile:
        tempfsf=infile.read()
        #
        if not os.path.exists(os.path.join(output_dir, "design_files")):
            os.makedirs(os.path.join(output_dir, "design_files"))
        #print(output_dir)
        design_fileout = os.path.join(output_dir, "design_files/%s_%s_%s_feat1.fsf"%(sub_id, ses_id, trial_id))
        out_param = data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id]["OUTPUT"]
        func_param = data_dict[sub_id][task]["FUNCRUN"]
        confound_param = data_dict[sub_id][task]["CONFOUND"]
        trial_param = data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id]["TRIAL"]
        nuis_param = data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id]["NUIS"]

        tempfsf = tempfsf.replace("OUTPUT", out_param)
        tempfsf = tempfsf.replace("FUNCRUN", func_param) 
        tempfsf = tempfsf.replace("CONFOUND", confound_param)
        tempfsf = tempfsf.replace("TRIAL", trial_param)
        tempfsf = tempfsf.replace("NUIS", nuis_param)

        for i in range(6):
            moco = data_dict[sub_id][task]["MOCO%i"%i]
            tempfsf = tempfsf.replace("MOCO%i"%i, moco)
        try:
            with open(design_fileout,'w') as outfile:
                outfile.write(tempfsf)
            outfile.close()
        except:
            print("BAD SUBJECT ", sub_id)
        infile.close()
        
        

In [3]:
def create_fsf(input_dir, deriv_dir, ses_id):
    
    ses_id=ses_id
    data_dict= {}
    # start loop -- looping through subjects
    subject_list = glob.glob(os.path.join('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives', 'sub-*/%s'%ses_id))
    for sub_path in subject_list:
        sub_id=sub_path.split("/")[-2]
        
        functional_tasks = glob.glob(os.path.join(sub_path, 'func/*milkshake*preproc_brain.nii.gz'))
        if sub_id not in data_dict:
            data_dict[sub_id] = {}
                
        
        for functional in functional_tasks: # SECOND LOOP -- looping through RUNS
            task=functional.split("/")[-1].split("_")[2].split("-")[1]
            analysis_folder=os.path.join('/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/%s/%s'%(sub_id,ses_id), "analysis")
            #print(analysis_folder)
            output_dir = os.path.join(analysis_folder, 'beta/task-%s'%task)
            
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            
            confound_file = os.path.join(sub_path, "func/motion_assessment/%s_%s_task-%s_bold_space-MNI152NLin2009cAsym_preproc_brain_confound.txt"%(sub_id,ses_id, task))
                
            data_dict[sub_id][task] = {
                "TRIALS" : { },
                "CONFOUND" : confound_file,
                "FUNCRUN" : functional
              }           
            
            for i in range(6):
                motcor=os.path.join(sub_path, 'func','motion_assessment', 'motion_parameters','%s_%s_task-%s_moco%s.txt'%(sub_id,ses_id, task,i))
                data_dict[sub_id][task]['MOCO%i'%i] = motcor
            
            
            subj_trials = sorted(glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/evs/%s*trial*.txt'%task.replace('milkshake', 'mk')))
            
            if not subj_trials:
                pass
            else:
                for trial_file in subj_trials:
                    _id = sub_id.split("-")[1]
                    _id = _id[1:]
                    trial_id = trial_file.split("/")[-1].split(".")[0]
                    #print(trial_id)
                    nuis_file = os.path.join('/projects/niblab/experiments/chocolate_milkshake/data/evs', '%s.txt'%trial_id.replace('trial', 'nuis'))
                    #print(nuis_file)
                    fileout = os.path.join(output_dir, "%s_%s_%s"%(sub_id, ses_id,trial_id))
                    #print(fileout)
                    data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id] = {"TRIAL" : trial_file, "NUIS": nuis_file, "OUTPUT" : fileout}
                    make_file(sub_id, ses_id, trial_id,output_dir, task, data_dict)
                    
                    
    return(data_dict)

In [4]:
def set_paths():
    #global input_dir
    #global deriv_dir
    print("[INFO] setting directory paths")
    input_dir =  '/projects/niblab/experiments/chocolate_milkshake'
        
    deriv_dir= os.path.join(input_dir, 'bids/derivatives')

    return input_dir, deriv_dir



In [5]:
def main():
    ses_id='ses-1'
    input_dir, deriv_dir=set_paths()
    data_dict=create_fsf(input_dir, deriv_dir,ses_id)
    
main()


[INFO] setting directory paths


## Run slurm jobs

In [89]:
def run_slurm(subject_list):
    for subj in subject_list[:1]:
        #print(subj)
        slurm_cmd= "sbatch --array={}%1 /projects/niblab/experiments/chocolate_milkshake/data/code/beta_run.job {}".format(subj,'receipt')
        print(slurm_cmd)
        #os.system(slurm_cmd)

In [90]:
def by_file_slurm(subject):
    fsfs=glob.glob(os.path.join('/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/{}/ses-1/analysis/beta/task*/design_files/*receipt_*.fsf'.format(subject)))
    for fsf in fsfs[1:]:
        slurm_cmd = "sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job {}".format(fsf)
        print('[INFO] submitted: \n', slurm_cmd)
        os.system(slurm_cmd)
        
        

Submit slurm files in a loop: 

In [91]:
subject_list =  [x.split("/")[-2].split("-")[1].strip('0') for x in glob.glob(os.path.join('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives', 'sub-*/ses-1'))]
run_slurm(subject_list)

sbatch --array=1%1 /projects/niblab/experiments/chocolate_milkshake/data/code/beta_run.job receipt


Submit slurm jobs by running fsf files individually:

In [92]:
sub_ids=[x.split('/')[-2] for x in glob.glob('/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-*/ses-1')]
for subject in sub_ids:
    print(subject)
    by_file_slurm(subject)

sub-001
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeC/design_files/sub-001_ses-1_mkC_HF_HS_receipt_trial2_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeC/design_files/sub-001_ses-1_mkC_HF_HS_receipt_trial3_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeC/design_files/sub-001_ses-1_mkC_HF_LS_receipt_trial1_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experime

[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-004/ses-1/analysis/beta/task-milkshakeB/design_files/sub-004_ses-1_mkB_LF_HS_receipt_trial2_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-004/ses-1/analysis/beta/task-milkshakeB/design_files/sub-004_ses-1_mkB_LF_HS_receipt_trial3_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-004/ses-1/analysis/beta/task-milkshakeB/design_files/sub-004_ses-1_mkB_LF_LS_receipt_trial1_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/choc

[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-005/ses-1/analysis/beta/task-milkshakeB/design_files/sub-005_ses-1_mkB_HF_HS_receipt_trial1_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-005/ses-1/analysis/beta/task-milkshakeB/design_files/sub-005_ses-1_mkB_HF_HS_receipt_trial2_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-005/ses-1/analysis/beta/task-milkshakeB/design_files/sub-005_ses-1_mkB_HF_HS_receipt_trial3_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/choc

[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-009/ses-1/analysis/beta/task-milkshakeB/design_files/sub-009_ses-1_mkB_LF_HS_receipt_trial2_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-009/ses-1/analysis/beta/task-milkshakeB/design_files/sub-009_ses-1_mkB_LF_HS_receipt_trial3_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-009/ses-1/analysis/beta/task-milkshakeB/design_files/sub-009_ses-1_mkB_LF_LS_receipt_trial1_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/choc

[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-012/ses-1/analysis/beta/task-milkshakeB/design_files/sub-012_ses-1_mkB_HF_LS_receipt_trial2_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-012/ses-1/analysis/beta/task-milkshakeB/design_files/sub-012_ses-1_mkB_HF_LS_receipt_trial3_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-012/ses-1/analysis/beta/task-milkshakeB/design_files/sub-012_ses-1_mkB_LF_HS_receipt_trial1_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/choc

...

[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-100/ses-1/analysis/beta/task-milkshakeA/design_files/sub-100_ses-1_mkA_HF_LS_receipt_trial3_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-100/ses-1/analysis/beta/task-milkshakeA/design_files/sub-100_ses-1_mkA_LF_HS_receipt_trial1_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-100/ses-1/analysis/beta/task-milkshakeA/design_files/sub-100_ses-1_mkA_LF_HS_receipt_trial2_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/choc

[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-111/ses-1/analysis/beta/task-milkshakeB/design_files/sub-111_ses-1_mkB_LF_HS_receipt_trial2_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-111/ses-1/analysis/beta/task-milkshakeB/design_files/sub-111_ses-1_mkB_LF_HS_receipt_trial3_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-111/ses-1/analysis/beta/task-milkshakeB/design_files/sub-111_ses-1_mkB_LF_LS_receipt_trial1_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/choc

[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-124/ses-1/analysis/beta/task-milkshakeA/design_files/sub-124_ses-1_mkA_LF_LS_receipt_trial3_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-124/ses-1/analysis/beta/task-milkshakeA/design_files/sub-124_ses-1_mkA_h20_receipt_trial1_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-124/ses-1/analysis/beta/task-milkshakeA/design_files/sub-124_ses-1_mkA_h20_receipt_trial2_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolat

[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-139/ses-1/analysis/beta/task-milkshakeA/design_files/sub-139_ses-1_mkA_h20_receipt_trial2_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-139/ses-1/analysis/beta/task-milkshakeA/design_files/sub-139_ses-1_mkA_h20_receipt_trial3_feat1.fsf
sub-146
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-146/ses-1/analysis/beta/task-milkshakeB/design_files/sub-146_ses-1_mkB_HF_HS_receipt_trial2_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/

[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-146/ses-1/analysis/beta/task-milkshakeA/design_files/sub-146_ses-1_mkA_h20_receipt_trial3_feat1.fsf
sub-147
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-147/ses-1/analysis/beta/task-milkshakeB/design_files/sub-147_ses-1_mkB_HF_HS_receipt_trial2_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-147/ses-1/analysis/beta/task-milkshakeB/design_files/sub-147_ses-1_mkB_HF_HS_receipt_trial3_feat1.fsf
[INFO] submitted: 
 sbatch /projects/niblab/experiments/chocolate_milkshake/data/code/beta_by_file.job /projects/niblab/experiment

View your job queue:

In [93]:
!squeue -u nbytes

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           3010908     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)
           3010909     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)
           3010910     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)
           3010911     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)
           3010912     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)
           3010913     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRA

           3011731     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)
           3011732     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)
           3011733     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)
           3011734     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)
           3011735     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)
           3011736     batch betaseri   nbytes PD       0:00      1 (Nodes required for job are DOWN, DRAINED or reserved for jobs in higher priority partitions)
           3011737     batch

## Quality Analysis


In [85]:
def quality_check():
    data_dict={}
    ses1_subjects=glob.glob(os.path.join("/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-*/ses-1"))
    for subj_folder in ses1_subjects:
        subject=subj_folder.split("/")[-2]
        if subject not in data_dict:
            data_dict[subject] = []
            
        feats=glob.glob(os.path.join(subj_folder, 'analysis/beta/task*/*.feat/'))
        if not feats:
            #print("[INFO] nothing found for %s"%subject)
            data_dict[subject] = "MISSING"
        else:
            for feat in feats:
                #print(feat)
                stats_folder=os.path.join(feat, "stats")
                #print(stats_folder)
                if not os.path.exists(stats_folder):
                    #print(feat)#.split("/")[-1])
                    #data_dict[subject].append(feat)
                    # remove folder if its bad/ no stats folder
                    rmtree(feat)
                    
                else:
                    #pass
                    #print("[INFO] passed")
                    #data_dict[subject]['good'].append(feat.split('/')[-2])
    #print(data_dict)
                    
                    

In [86]:
quality_check()

[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO]

[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO]

[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO]

[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO]

[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO]

[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO] passed
[INFO]